In [1]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import os
import random
from tqdm.keras import TqdmCallback

# 설정
IMAGE_SIZE = 160   # 줄임
BATCH_SIZE = 32
EPOCHS = 3       # 에포크는 유지, 얼리스톱핑으로 조절
NUM_CLASSES = 7
train_dir = './open/train'

# 데이터프레임 기반 구성 (샘플링 10000장으로 제한)
all_data = []
for class_name in sorted(os.listdir(train_dir)):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        images = [img for img in images if img.lower().endswith('.jpg')]
        sampled_images = random.sample(images, min(10000, len(images)))
        for img in sampled_images:
            all_data.append({'filename': os.path.join(class_path, img), 'class': class_name})

df = pd.DataFrame(all_data)
df = df.sample(frac=1).reset_index(drop=True)

from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['class'], random_state=42)


In [2]:
# ImageDataGenerator (augmentation 최소화, validation_split 제거)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,   # 줄임
    zoom_range=0.1,      # 줄임
    width_shift_range=0.05,
    height_shift_range=0.05
)

val_datagen = ImageDataGenerator(rescale=1./255)

# flow_from_dataframe 사용 (이미지 경로와 라벨 직접 지정 가능)
train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='class',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 56000 validated image filenames belonging to 7 classes.
Found 14000 validated image filenames belonging to 7 classes.


In [3]:
# 모델 딕셔너리
model_classes = {
    'MobileNetV3Small': tf.keras.applications.MobileNetV3Small,
    'MobileNetV2': tf.keras.applications.MobileNetV2,
    'EfficientNetV2B0': tf.keras.applications.EfficientNetV2B0,
    'NASNetMobile': tf.keras.applications.NASNetMobile
}

def build_model(base_model_class):
    base = base_model_class(include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet')
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(0.3)(x)
    output = Dense(NUM_CLASSES, activation='softmax')(x)
    model = Model(base.input, output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)


In [4]:
results = {}

for name, model_class in model_classes.items():
    print(f"\n▶ Training {name} ...")
    model = build_model(model_class)
    model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        verbose=0,              # tqdm 콜백 쓸 때는 0으로
        callbacks=[TqdmCallback(verbose=1), early_stop]
    )
    loss, acc = model.evaluate(val_gen, verbose=0)
    results[name] = acc
    print(f"{name} validation accuracy: {acc:.4f}")

print("\n📊 Final Validation Accuracy Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")
    


▶ Training MobileNetV3Small ...


C:\Users\FOR\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\applications\mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

C:\Users\FOR\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

0batch [00:00, ?batch/s]

Restoring model weights from the end of the best epoch: 1.
EfficientNetV2B0 validation accuracy: 0.1429

▶ Training NASNetMobile ...
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Restoring model weights from the end of the best epoch: 2.
NASNetMobile validation accuracy: 0.1967

📊 Final Validation Accuracy Results:
MobileNetV3Small: 0.1429
MobileNetV2: 0.5024
EfficientNetV2B0: 0.1429
NASNetMobile: 0.1967
